# Noise Generator
### Author: Cathal Ó Faoláin
### 15:47, 02/10/2024

The goal of this work is to understand how we can use predicted IHC potentials, such as those predicted by WavIHC, introduced in the paper "WaveNet-based approximation of a cochlear filtering and hair cell transduction model".  Feature encoders designed to use these predicted IHC potentials are evaluated against other state-of-the-art feature encoders in order to understand how discriminating they are, and over a range of different Signal-to-Noise Ratios (SNRs).

This Notebook generates noise over a range of [30 to -10] dB SNRs, going in steps of 5dB. These are then added to the TIMIT test dataset for use with our phoneme discrimination models. 

In [1]:
import torch
from torch import nn
import librosa
import time
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from torch.utils.data import DataLoader, Dataset, IterableDataset
import torchaudio
import pandas as pd
import numpy as np
import time
import sys
import yaml
import math
import scipy.signal as signal
from dataclasses import dataclass, field
from typing import List, Tuple
import torch.nn.functional as F
from pathlib import Path
import pickle
import soundfile as sf
import IPython.display as ipd
import glob
import random
import json
import math
import audioread

In [2]:
audioread.available_backends()

[audioread.rawread.RawAudioFile, audioread.ffdec.FFmpegAudioFile]

In [3]:
sys.path.append('./IHCApproxNH/')
from classes import WaveNet
from utils import utils
from Encoders import FeatureEncoders 
from TIMIT_utils import TIMIT_utils
#from Train_TestFunctions import TrainEvalFunctions

In [4]:
#Set the paths to the different datasets
train_path=Path('Data/TIMIT_Metadata_Train.csv')
validation_path=Path('Data/TIMIT_Metadata_Validation.csv')
test_path=Path('Data/TIMIT_Metadata_Test.csv')
kfold_paths=[" Data/kFolds/TIMIT_Metadata_Train_k1.csv",  " Data/kFolds/TIMIT_Metadata_Train_k2.csv", " Data/kFolds/TIMIT_Metadata_Train_k3.csv", " Data/kFolds/TIMIT_Metadata_Train_k4.csv", " Data/kFolds/TIMIT_Metadata_Train_k5.csv"]

#Set the values of SNR that we are going to add
SNRs=[30, 25, 20, 15, 10, 5, 0, -5, -10]
fs=16000

#Make the Noisy directory 
dir_noise = Path("./Data/Noisy")
dir_noise.mkdir(parents=True, exist_ok=True)
print("Making Directory: %s" % dir_noise)

Making Directory: Data/Noisy


## Noise Test Addition: Steady Noise
### White Noise:
This adds white noise to the TIMIT test dataset, to allow for testing our models over a range of SNRs.

In [5]:
#This function adds white noise at a given snr
def add_white_noise(signal, snr_db):
    signal_power=np.sum(signal**2)/len(signal)

    snr=10**(snr_db/10)
    noise_power=signal_power / snr
    
    noise=np.random.normal(0, math.sqrt(noise_power), signal.size)
    noisy_signal=signal+noise

    return noisy_signal

In [6]:
#Loop through the snrs, adding noise to each file
test_df=pd.read_csv(test_path)
test_df=test_df.drop(columns="Unnamed: 0")

#Make the white noise directory
dir_white = Path("./Data/Noisy/White/")
dir_white.mkdir(parents=True, exist_ok=True)
print("Making Directory: %s" % dir_white)

#For each SNR 
for n in SNRs:
    #Make the SNR noise directory
    dir_snr = Path("./Data/Noisy/White/{}".format(n))
    dir_snr.mkdir(parents=True, exist_ok=True)
    print("Making Directory: %s" % dir_snr)
    
    #Make a df to store the values, with the same columns as the original test_df
    noisy_df=pd.DataFrame(columns=test_df.columns)

    #for each file
    for idx, file in test_df.iterrows():
        #Get the audio file path to save, splitting on the . just before the file
        path=file['Audio'].split(".WAV")
        #Make sure the file save path exists
        file_dir=Path(path[0])
        file_dir.mkdir(parents=True, exist_ok=True)
        noise_dir=Path(path[0] +"/white")
        noise_dir.mkdir(parents=True, exist_ok=True)
        save_path=path[0] + "/white/SNR_{}.WAV".format(n)
        
        #Read in the audio file and add noise to it
        signal, sampling=sf.read(file['Audio'])
        noisy_signal=add_white_noise(signal, n)

        #Save the new file
        sf.write(save_path, noisy_signal, sampling)

        #Open the new wav file and check the sampling rate. This should throw an error 
        #if something went wrong
        data, fsaved=librosa.load(save_path, sr=None)
        assert(sampling==fsaved)
        saved_file=Path(save_path)
        assert(saved_file.is_file())
        

        #Save the generated audio file in the audio position
        file['Audio']=saved_file
        noisy_df=noisy_df._append(file, ignore_index=True)
        
    

    print("Saving noisy metadata dataset to: Data/Noisy/White/{}/TIMIT_Metadata_Test.csv".format(n)) 
    noisy_df.to_csv('Data/Noisy/White/{}/TIMIT_Metadata_Test.csv'.format(n))
    
    #Make sure that we made the file correctly. There should be no NaNs, should be the same length 
    file=Path('Data/Noisy/White/{}/TIMIT_Metadata_Test.csv'.format(n))
    assert(file.is_file())
    assert(len(test_df)==len(noisy_df))
    assert(noisy_df.isnull().any().any()==False)


#Show an example of what the noisy df looks like
print(noisy_df)
#Show a sample of the new sound
print("Taking audio sample from Data/Noisy/White/{}/TIMIT_Metadata_Test.csv".format(n))
ipd.Audio(noisy_df['Audio'][0])

Making Directory: Data/Noisy/White
Making Directory: Data/Noisy/White/30
Saving noisy metadata dataset to: Data/Noisy/White/30/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/White/25
Saving noisy metadata dataset to: Data/Noisy/White/25/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/White/20
Saving noisy metadata dataset to: Data/Noisy/White/20/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/White/15
Saving noisy metadata dataset to: Data/Noisy/White/15/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/White/10
Saving noisy metadata dataset to: Data/Noisy/White/10/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/White/5
Saving noisy metadata dataset to: Data/Noisy/White/5/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/White/0
Saving noisy metadata dataset to: Data/Noisy/White/0/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/White/-5
Saving noisy metadata dataset to: Data/Noisy/White/-5/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/White/-10
Sa

In [7]:
#Show a sample of the new sound
print("Taking audio sample from Data/Noisy/White/30/TIMIT_Metadata_Test.csv")
noisy_df=pd.read_csv("./Data/Noisy/White/30/TIMIT_Metadata_Test.csv")
ipd.Audio(noisy_df['Audio'][0])

Taking audio sample from Data/Noisy/White/30/TIMIT_Metadata_Test.csv


## Noise k-Fold Add
Here we will add white noise to each of the k-folds. The white noise will be added at a random snr each time, and this should facilitate fine tuning for noise.

In [8]:
#This function adds white noise at a given snr
def add_white_noise(signal, snr_db):
    signal_power=np.sum(signal**2)/len(signal)

    snr=10**(snr_db/10)
    noise_power=signal_power / snr
    
    noise=np.random.normal(0, math.sqrt(noise_power), signal.size)
    noisy_signal=signal+noise

    return noisy_signal

In [9]:
#This function selects a random snr for a given noise type
class random_Noise():
    def __init__(self):
        """
        This function initialises a random seed to ensure reproducibility and sets the snr list that we can pick from.
        """
        
        #Set up the allowed SNRs for reproducibility
        self.SNRs=SNRs=[30, 25, 20, 15, 10, 5, 0, -5, -10]

        #Set up a random seed to ensure reproducibility
        random.seed(101)

    def pick_add_SNR(self, signal, noise):
        """
        this adds noise at a random SNR based on those that are allowed
        """
        #Pick a random snr
        snr_db=random.choice(self.SNRs)

        #Add white noise if that is the noise type
        if(noise=="White"):
            #returns the noisy signal
            return add_white_noise(signal, snr_db)
    
    

In [10]:
#Make the white noise directory
dir_white = Path("./Data/Noisy/White/kFolds")
dir_white.mkdir(parents=True, exist_ok=True)
print("Making Directory: %s" % dir_white)

#Loop through all the kfolds adding in noise as we go
kfold_paths=["./Data/kFolds/TIMIT_Metadata_Train_k1.csv",  "./Data/kFolds/TIMIT_Metadata_Train_k2.csv", "./Data/kFolds/TIMIT_Metadata_Train_k3.csv", "./Data/kFolds/TIMIT_Metadata_Train_k4.csv", "./Data/kFolds/TIMIT_Metadata_Train_k5.csv"]

#Set the seed for adding the noise at a random SNR
Snr_adder=random_Noise()

#For each kfold
for k in kfold_paths:
    test_df=pd.read_csv(k)

    file_locat=k.split('/')[-1]
    print(file_locat)
    test_df=test_df.drop(columns="Unnamed: 0")

    #Make a df to store the values, with the same columns as the original kfold dataframe
    noisy_df=pd.DataFrame(columns=test_df.columns)

    #for each file
    for idx, file in test_df.iterrows():
        #Get the audio file path to save, splitting on the . just before the file
        path=file['Audio'].split(".WAV")
        #Make sure the file save path exists
        file_dir=Path(path[0])
        file_dir.mkdir(parents=True, exist_ok=True)
        noise_dir=Path(path[0] +"/white")
        noise_dir.mkdir(parents=True, exist_ok=True)
        save_path=path[0] + "/white/kFold.WAV".format(n)
        
        #Read in the audio file and add noise to it
        signal, sampling=sf.read(file['Audio'])
        noisy_signal=Snr_adder.pick_add_SNR(signal, "White")

        #Save the new file
        sf.write(save_path, noisy_signal, sampling)

        #Open the new wav file and check the sampling rate. This should throw an error 
        #if something went wrong
        data, fsaved=librosa.load(save_path, sr=None)
        assert(sampling==fsaved)
        saved_file=Path(save_path)
        assert(saved_file.is_file())
        

        #Save the generated audio file in the audio position
        file['Audio']=saved_file
        noisy_df=noisy_df._append(file, ignore_index=True)
        
    
    
    print("Saving noisy metadata dataset to: Data/Noisy/White/kFolds/{}".format(file_locat)) 
    noisy_df.to_csv('Data/Noisy/White/kFolds/{}'.format(file_locat))
    
    #Make sure that we made the file correctly. There should be no NaNs, should be the same length 
    file=Path('Data/Noisy/White/kFolds/{}'.format(file_locat))
    assert(file.is_file())
    assert(len(test_df)==len(noisy_df))
    assert(noisy_df.isnull().any().any()==False)


#Show an example of what the noisy df looks like
print(noisy_df)
#Show a sample of the new sound
print("Taking audio sample from Data/Noisy/White/kFolds/{}".format(file_locat))
ipd.Audio(noisy_df['Audio'][0])

Making Directory: Data/Noisy/White/kFolds
TIMIT_Metadata_Train_k1.csv
Saving noisy metadata dataset to: Data/Noisy/White/kFolds/TIMIT_Metadata_Train_k1.csv
TIMIT_Metadata_Train_k2.csv
Saving noisy metadata dataset to: Data/Noisy/White/kFolds/TIMIT_Metadata_Train_k2.csv
TIMIT_Metadata_Train_k3.csv
Saving noisy metadata dataset to: Data/Noisy/White/kFolds/TIMIT_Metadata_Train_k3.csv
TIMIT_Metadata_Train_k4.csv
Saving noisy metadata dataset to: Data/Noisy/White/kFolds/TIMIT_Metadata_Train_k4.csv
TIMIT_Metadata_Train_k5.csv
Saving noisy metadata dataset to: Data/Noisy/White/kFolds/TIMIT_Metadata_Train_k5.csv
      idx                                              Audio  \
0    4953  Data/timit/TIMIT/TEST/DR7/MDVC0/SX396_converte...   
1    5991  Data/timit/TIMIT/TEST/DR5/MRPP0/SI1814_convert...   
2    3507  Data/timit/TIMIT/TRAIN/DR8/FBCG1/SX82_converte...   
3    1080  Data/timit/TIMIT/TRAIN/DR7/FCRZ0/SX73_converte...   
4    6295  Data/timit/TIMIT/TEST/DR1/FJEM0/SX184_converte...   
..  

## Noise Test Add: Amplitude Modulated

This should add noise from the Urban Sounds dataset to each of the TIMIT speech files available at the different SNRs. This is amplitue modulated, as it is naturally occurring. We have noise of 10 classes available to us:
- Air Conditioner
- Children Playing
- Dog Barking
- Street music
- Engine idling
- Jackhammer
- Siren
- Gunshot
- Car horn

We will initially add noise from the first 5 noise classes to the test set, at the required SNRs.

In [11]:
noise_types=["air_conditioner","car_horn", "gun_shot", "children_playing",
             "dog_bark", "drilling", "engine_idling", "jackhammer", 
            "siren", "street_music"]

In [12]:
def get_durations(df):
    total_durations = 0

    for idx, entry in df.iterrows():
        audio_data, fs = librosa.load(entry['Audio'], sr=16000)
        duration = len(audio_data) / fs
        total_durations += duration
    
    average=total_durations/len(df)
    return total_durations, average

In [13]:
#This object adds amplitude modulated noise at a given snr, taken from the urban soundnoise dataset
class Noise_Adder:
    def __init__(self, noise):
        """
        This initialises the noise adder class. A list of all the noise files for this type is collected, 
        the random seed is selected (to ensure reproducibility), and a random initial file is selected from 
        file list. The time in the file is set to 0
        """
        #Get the path to the desired noise files
        noise_path="./Data/UrbanSound/data/{}".format(noise)
        self.path=noise_path
                        
        self.noise=noise
        print("*** INTIALISING NOISE ADDER FOR NOISE of type : " + noise + " ***")
        
        self.current=None
        self.file_count=0;
        #Create a list of every audio file an associated csv - we will store that
        
        self.file_list=glob.glob(noise_path+ '/*.csv')

        #Set a random file to current. Makes sure that the files are always the same too
        random.seed(1079)
        self.randomFile()
        
                   
    def enoughTime(self, test_df):
        """
        This function simply checks if there is enough noise files that could be added to the dataset
        that are of the same length or longer than the average TIMIT sound file.
        
        Inputs:
        self
        self.file_list
        """
        #Get the total and average duration of the test df
        total_dur, avg_dur=get_durations(test_df)
        
        total_durations = 0

        for entry in self.file_list:
            noiseData=pd.read_csv(entry, header=None)
            #This corresponds to end time - start time based on the UrbanSound README
            duration = noiseData[1][0]-noiseData[0][0]
            if(duration > avg_dur):
                total_durations += duration
            
        print("TOTAL CLEAN TIME || TOTAL NOISY TIME")
        print("   {} mins       ||        {} mins".format(int(total_dur)//60, int(total_durations)//60))
        print("Enough time to add to every file : ")
        print(total_dur < total_durations)
        print(" ")
        
        return total_dur < total_durations
                        
    def randomFile(self):
        """
        This returns a psuedo-random new file.
        
        Inputs:
        
        Self,
        self.current,
        self.file_list
        """
        #If there is a value for self current, remove it from the list
        if self.current!= None:
            self.file_list.remove(self.current)
        
        #Get a random choice from the file list
        try:
            self.current=random.choice(self.file_list)
            self.time=0;
        except:
            print("ERROR: ran out of examples of class " + self.noise)
           
    def timeRemaining(self, signal):
        """
        This checks if there is any space left in the noise file to be used in adding. There needs to be 
        enough noise to be present to be added to the entire length of the signal
        """
        #Read in the current csv file, including our added "time added"
        noiseData=pd.read_csv(self.current, header=None)
        
        #These come from the Urban Sounds README
        col_meaning={0:"start time", 1:"end time", 2:"salience label", 3:"class label"}
        noiseData=noiseData.rename(columns=col_meaning)
        
        #Make sure that the time in the file is at least the start time
        if (self.time < noiseData["start time"][0]):
            self.time=math.ceil(noiseData["start time"][0]*16000)
            
        time_left=math.ceil(noiseData["end time"][0]*16000) - self.time
        
        #Make sure there is enough time left to fill the signal time. Return the time multiplied by
        #The sampling rate we are using
        return time_left
        
    def loadFile(self):
        """
        This loads in the noise file. This is slightly more complicated on UrbanSounds, as the codec
        can vary from sound to sound. However, each file has a json. We will read that, get the codec,
        change the audio to mono if required and return the file
        """
        #Go from csv to json.
        filename=self.current
        fileStem=filename.split(".csv")[0]
        
        jsonfile= fileStem + ".json"
        
        #Open the json file
        f=open(jsonfile, "r")
        
        data=json.load(f)
        f.close()
        
        #Get the codec and use it to load the file
        audiofile=Path(fileStem+"." +data['type'])
        
        #Make sure the noise file is also at 16kHz
        try:
            noise, sr=librosa.load(audiofile, sr=16000)
        except:
            print("ERROR reading in " + str(audiofile) +". Likely due to lack of codec support"+ \
            "(no backend). Trying with soundfile")
            noise, sr=sf.read(audiofile)

            print(sr)
                
        
        
        noise=noise.reshape((1, len(noise)))
        
        #Get a mono version of the audiofile
        noise=np.mean(noise, axis=0, keepdims=True)
        noise=np.squeeze(noise, axis=0)
        
        return noise
        
                        
    def add_real_noise(self, signal, snr_db):
        """
        This is the method that adds noise to the file at the required snr
        """
        #Make sure we have a file that is long enough to be mixed into the signal
        while(1):
            time_left=self.timeRemaining(signal)
            
            #Break if there is enough time left, otherwise get the next file
            if(time_left> len(signal)):
                   break
            else:
                self.randomFile()
            
        #Get the noise file
        noise=self.loadFile()
        
        #First we will clip the noise file to be as long as the signal, in the noisy part of the signal
        noise=noise[self.time : self.time+len(signal)]
        #Update the time left in this noise file
        self.time+=len(signal)
    
        #ADD THE NOISE AT THE REQUIRED LEVEL
        #Get the power of the two values
        signal_power=np.sum(signal**2)/len(signal)
        noise_power=np.sum(noise**2)/len(noise)

        #Undo the logarithmic to get the scale
        snr=10**(snr_db/10)
        #Get the factor by which to scale the output
        try:
            #Get the alpha we are going to need to scale the noise by
            alpha=math.sqrt(signal_power/(snr*noise_power))
        except:
            print(len(noise_power))
            print(len(signal_power))
            print(snr)
    
        noisy_signal=signal+alpha*noise

        return noisy_signal

In [14]:
#Loop through the snrs, adding noise to each file
test_df=pd.read_csv(test_path)
test_df=test_df.drop(columns="Unnamed: 0")

usable_noise=[]
#Make sure that we only use datasets that can 
for noise in noise_types:
    adder=Noise_Adder(noise)
    
    if( adder.enoughTime(test_df)):
        usable_noise.append(noise)
        
print(usable_noise)

*** INTIALISING NOISE ADDER FOR NOISE of type : air_conditioner ***
TOTAL CLEAN TIME || TOTAL NOISY TIME
   48 mins       ||        85 mins
Enough time to add to every file : 
True
 
*** INTIALISING NOISE ADDER FOR NOISE of type : car_horn ***
TOTAL CLEAN TIME || TOTAL NOISY TIME
   48 mins       ||        3 mins
Enough time to add to every file : 
False
 
*** INTIALISING NOISE ADDER FOR NOISE of type : gun_shot ***
TOTAL CLEAN TIME || TOTAL NOISY TIME
   48 mins       ||        0 mins
Enough time to add to every file : 
False
 
*** INTIALISING NOISE ADDER FOR NOISE of type : children_playing ***
TOTAL CLEAN TIME || TOTAL NOISY TIME
   48 mins       ||        282 mins
Enough time to add to every file : 
True
 
*** INTIALISING NOISE ADDER FOR NOISE of type : dog_bark ***
TOTAL CLEAN TIME || TOTAL NOISY TIME
   48 mins       ||        101 mins
Enough time to add to every file : 
True
 
*** INTIALISING NOISE ADDER FOR NOISE of type : drilling ***
TOTAL CLEAN TIME || TOTAL NOISY TIME
   48

In [15]:
noise_types=['air_conditioner', 'dog_bark', 'street_music', 'children_playing']

In [16]:
#Loop through the snrs, adding noise to each file
test_df=pd.read_csv(test_path)
test_df=test_df.drop(columns="Unnamed: 0")

for noise in noise_types:
    #Make the associated noise directory
    dir_white = Path("./Data/Noisy/{}/".format(noise))
    dir_white.mkdir(parents=True, exist_ok=True)
    print("Making Directory: %s" % dir_white)

    #For each SNR 
    for n in SNRs:
        #Make the Noise Adder
        adder=Noise_Adder(noise)
        
        #Make the SNR noise directory
        dir_snr = Path("./Data/Noisy/{}/{}".format(noise, n))
        dir_snr.mkdir(parents=True, exist_ok=True)
        print("Making Directory: %s" % dir_snr)
    
        #Make a df to store the values, with the same columns as the original test_df
        noisy_df=pd.DataFrame(columns=test_df.columns)

        #for each TIMIT file
        for idx, file in test_df.iterrows():
            #Get the audio file path to save, splitting on the . just before the file
            path=file['Audio'].split(".WAV")
            #Make sure the file save path exists
            file_dir=Path(path[0])
            file_dir.mkdir(parents=True, exist_ok=True)
            noise_dir=Path(path[0] +"/{}".format(noise))
            noise_dir.mkdir(parents=True, exist_ok=True)
            save_path=path[0] + "/{}/SNR_{}.WAV".format(noise, n)
        
            #Read in the audio file and add noise to it
            signal, sampling=sf.read(file['Audio'])
            noisy_signal=adder.add_real_noise(signal, n)

            #Save the new file
            sf.write(save_path, noisy_signal, sampling)

            #Open the new wav file and check the sampling rate. This should throw an error 
            #if something went wrong
            data, fsaved=librosa.load(save_path, sr=None)
            assert(sampling==fsaved)
            saved_file=Path(save_path)
            assert(saved_file.is_file())
        

            #Save the generated audio file in the audio position
            file['Audio']=saved_file
            noisy_df=noisy_df._append(file, ignore_index=True)
    
    
        print("Saving noisy metadata dataset to: Data/Noisy/{}/{}/TIMIT_Metadata_Test.csv".format(noise, n)) 
        noisy_df.to_csv('Data/Noisy/{}/{}/TIMIT_Metadata_Test.csv'.format(noise, n))
    
        #Make sure that we made the file correctly. There should be no NaNs, should be the same length 
        file=Path('Data/Noisy/{}/{}/TIMIT_Metadata_Test.csv'.format(noise, n))
        assert(file.is_file())
        assert(len(test_df)==len(noisy_df))
        assert(noisy_df.isnull().any().any()==False)


#Show an example of what the noisy df looks like
print(noisy_df)
#Show a sample of the new sound
print("Taking audio sample from Data/Noisy/{}/{}/TIMIT_Metadata_Test.csv".format(noise, n))
ipd.Audio(noisy_df['Audio'][0])

Making Directory: Data/Noisy/air_conditioner
*** INTIALISING NOISE ADDER FOR NOISE of type : air_conditioner ***
Making Directory: Data/Noisy/air_conditioner/30


/tmp/ipykernel_3058790/1077044784.py:122: UserWarning: PySoundFile failed. Trying audioread instead.
  noise, sr=librosa.load(audiofile, sr=16000)
/home/cathaloF/miniconda3/envs/eist/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_3058790/1077044784.py:122: UserWarning: PySoundFile failed. Trying audioread instead.
  noise, sr=librosa.load(audiofile, sr=16000)
/home/cathaloF/miniconda3/envs/eist/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_3058790/1077044784.py:122: UserWarning: PySoundFile failed. Trying audioread instead.
  noise, sr=libro

Saving noisy metadata dataset to: Data/Noisy/air_conditioner/30/TIMIT_Metadata_Test.csv
*** INTIALISING NOISE ADDER FOR NOISE of type : air_conditioner ***
Making Directory: Data/Noisy/air_conditioner/25


/tmp/ipykernel_3058790/1077044784.py:122: UserWarning: PySoundFile failed. Trying audioread instead.
  noise, sr=librosa.load(audiofile, sr=16000)
/home/cathaloF/miniconda3/envs/eist/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_3058790/1077044784.py:122: UserWarning: PySoundFile failed. Trying audioread instead.
  noise, sr=librosa.load(audiofile, sr=16000)
/home/cathaloF/miniconda3/envs/eist/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_3058790/1077044784.py:122: UserWarning: PySoundFile failed. Trying audioread instead.
  noise, sr=libro

Saving noisy metadata dataset to: Data/Noisy/air_conditioner/25/TIMIT_Metadata_Test.csv
*** INTIALISING NOISE ADDER FOR NOISE of type : air_conditioner ***
Making Directory: Data/Noisy/air_conditioner/20


...

Saving noisy metadata dataset to: Data/Noisy/dog_bark/0/TIMIT_Metadata_Test.csv
*** INTIALISING NOISE ADDER FOR NOISE of type : dog_bark ***
Making Directory: Data/Noisy/dog_bark/-5


[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping
[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping
[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping
[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping
[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping


Saving noisy metadata dataset to: Data/Noisy/dog_bark/-5/TIMIT_Metadata_Test.csv
*** INTIALISING NOISE ADDER FOR NOISE of type : dog_bark ***
Making Directory: Data/Noisy/dog_bark/-10


[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping
[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping
[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping
[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping
[src/libmpg123/id3.c:INT123_parse_new_id3():947] warning: ID3v2: unrealistic small tag lengh 0, skipping


Saving noisy metadata dataset to: Data/Noisy/dog_bark/-10/TIMIT_Metadata_Test.csv
Making Directory: Data/Noisy/street_music
*** INTIALISING NOISE ADDER FOR NOISE of type : street_music ***
Making Directory: Data/Noisy/street_music/30
Saving noisy metadata dataset to: Data/Noisy/street_music/30/TIMIT_Metadata_Test.csv
*** INTIALISING NOISE ADDER FOR NOISE of type : street_music ***
Making Directory: Data/Noisy/street_music/25
Saving noisy metadata dataset to: Data/Noisy/street_music/25/TIMIT_Metadata_Test.csv
*** INTIALISING NOISE ADDER FOR NOISE of type : street_music ***
Making Directory: Data/Noisy/street_music/20
Saving noisy metadata dataset to: Data/Noisy/street_music/20/TIMIT_Metadata_Test.csv
*** INTIALISING NOISE ADDER FOR NOISE of type : street_music ***
Making Directory: Data/Noisy/street_music/15
Saving noisy metadata dataset to: Data/Noisy/street_music/15/TIMIT_Metadata_Test.csv
*** INTIALISING NOISE ADDER FOR NOISE of type : street_music ***
Making Directory: Data/Noisy/st

In [17]:
#Show a sample of the new sound
print("Taking audio sample from Data/Noisy/air_conditioner/-10/TIMIT_Metadata_Test.csv")
noisy_df=pd.read_csv("./Data/Noisy/air_conditioner/-10/TIMIT_Metadata_Test.csv")
ipd.Audio(noisy_df['Audio'][0])

Taking audio sample from Data/Noisy/air_conditioner/-10/TIMIT_Metadata_Test.csv


In [18]:
#Show a sample of the new sound
print("Taking audio sample from Data/Noisy/dog_bark/5/TIMIT_Metadata_Test.csv")
noisy_df=pd.read_csv("./Data/Noisy/dog_bark/5/TIMIT_Metadata_Test.csv")
ipd.Audio(noisy_df['Audio'][0])

Taking audio sample from Data/Noisy/dog_bark/5/TIMIT_Metadata_Test.csv


In [19]:
#Show a sample of the new sound
print("Taking audio sample from Data/Noisy/dog_bark/-10/TIMIT_Metadata_Test.csv")
noisy_df=pd.read_csv("./Data/Noisy/dog_bark/-10/TIMIT_Metadata_Test.csv")
ipd.Audio(noisy_df['Audio'][0])

Taking audio sample from Data/Noisy/dog_bark/-10/TIMIT_Metadata_Test.csv


In [20]:
#Show a sample of the new sound
print("Taking audio sample from Data/Noisy/children_playing/-10/TIMIT_Metadata_Test.csv")
noisy_df=pd.read_csv("./Data/Noisy/children_playing/-10/TIMIT_Metadata_Test.csv")
ipd.Audio(noisy_df['Audio'][0])

Taking audio sample from Data/Noisy/children_playing/-10/TIMIT_Metadata_Test.csv
